In [ ]:
%matplotlib inline

import pandas as pd
import numpy as np
import common_libs.utilities as ut
import matplotlib.pyplot as plt
import torch

In [ ]:
t = pd.read_sql('SELECT * FROM time JOIN time_metadata ON time.time_id=time_metadata.time_id WHERE kind_id=1', cnx = ut.create_connection()).set_index('time_id')

In [ ]:
good_times = t[(t['l1drmisses'] <= 0) & (t['l1dwmisses'] <= 0) & (t['l1imisses'] <= 0) & (t['conswitch'] <= 0)]

In [ ]:
def percentile(k):
    return lambda a: np.percentile(a, k)

In [ ]:
summaries = good_times.groupby('code_id').agg({'code_id': 'count', 'cycle_count': percentile(20)})

In [ ]:
true_cycles = summaries[summaries['code_id'] > 1]['cycle_count']

In [ ]:
data = torch.load('../saved/data_040519.data')
test_code_ids = set([d[0] for d in data[int(len(data)*.8):]])

In [ ]:
t_intel = pd.read_sql('SELECT * FROM time WHERE kind_id=2', ut.create_connection()).groupby('code_id').mean()['cycle_count']
err = ((true_cycles - t_intel).abs() / true_cycles).dropna()
test_err = ((true_cycles - t_intel[t_intel.index.isin(test_code_ids)]).abs() / true_cycles).dropna()
print('Intel: {}/{} ({:.0f}%) blocks\nAverage error {}\n{}/{} ({:.0f}%) test blocks\nAverage error {}'.format(
    len(err), len(true_cycles.dropna()), 100 * (float(len(err)) / len(true_cycles.dropna())),
    err.mean(),
    len(test_err), len(test_code_ids), 100 * (float(len(test_err)) / len(test_code_ids)),
    test_err.mean()
))

In [ ]:
t_llvm = pd.read_sql('SELECT * FROM time WHERE kind_id=3', ut.create_connection()).groupby('code_id').mean()['cycle_count']
err = ((true_cycles - t_llvm).abs() / true_cycles).dropna()
test_err = ((true_cycles - t_llvm[t_llvm.index.isin(test_code_ids)]).abs() / true_cycles).dropna()
print('LLVM: {}/{} ({:.0f}%) blocks\nAverage error {}\n{}/{} ({:.0f}%) test blocks\nAverage error {}'.format(
    len(err), len(true_cycles.dropna()), 100 * (float(len(err)) / len(true_cycles.dropna())),
    err.mean(),
    len(test_err), len(test_code_ids), 100 * (float(len(test_err)) / len(test_code_ids)),
    test_err.mean()
))